## Final

In [52]:
from deepface import DeepFace
import cv2
import os
import pyttsx3  # Import the text-to-speech library
import time  # Import time for managing the greeting intervals

# Define backends, models, and metrics
backends = ["opencv", "ssd", "dlib", "mtcnn", "retinaface"]
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
metrics = ["cosine", "euclidean", "euclidean_l2"]

# Initialize the text-to-speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Speed of speech
engine.setProperty('volume', 1)  # Volume level (0.0 to 1.0)

def calculate_accuracy(distance, metric):
    if metric == "cosine":
        accuracy = (1 - distance) * 100
    elif metric in ["euclidean", "euclidean_l2"]:
        threshold = 1.0  # You can adjust this threshold based on your requirements
        accuracy = (1 - distance / threshold) * 100
    return round(accuracy, 2)

def greet_person(name):
    greeting_message = f"Hello {name}, Welcome to Nine Pointers!"
    engine.say(greeting_message)
    engine.runAndWait()

def greet_unknown():
    greeting_message = "Welcome to Nine Pointers!"
    engine.say(greeting_message)
    engine.runAndWait()

def greet_group():
    greeting_message = "yaamada kuda sai"
    engine.say(greeting_message)
    engine.runAndWait()

def realtime_face_recognition():
    vid = cv2.VideoCapture(0)
    
    # Set video frame width and height for better performance
    vid.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    last_greeted_person = None
    last_greeted_time_person = 0
    last_greeted_time_unknown = 0
    
    while True:
        ret, frame = vid.read()
        if not ret:
            break
        
        # Resize frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        
        try:
            # Perform face recognition
            results = DeepFace.find(img_path=small_frame, db_path="img/", model_name=models[2], distance_metric=metrics[0], enforce_detection=False, detector_backend=backends[0])
            
            # Convert results to a list of DataFrames
            people = []
            if isinstance(results, list):
                for result in results:
                    people.append(result)
            else:
                people.append(results)
            
            recognized = False  # Flag to check if any face is recognized
            recognized_people = []  # List of recognized people names
            
            for person in people:
                if person.empty:
                    continue
                
                # Extract the first row of the DataFrame
                person = person.iloc[0]
                
                # Get the bounding box coordinates
                x = int(person['source_x'] * 2)
                y = int(person['source_y'] * 2)
                w = int(person['source_w'] * 2)
                h = int(person['source_h'] * 2)
                
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                # Extract the folder name (person's name)
                name = os.path.basename(os.path.dirname(person['identity']))
                distance = person['distance']
                accuracy = calculate_accuracy(distance, metrics[0])
                
                cv2.putText(frame, f"{name} ({accuracy}%)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                
                recognized_people.append(name)
                recognized = True
            
            people_count = len(recognized_people)
            
            if people_count > 1 and time.time() - last_greeted_time_unknown >= 15:
                greet_group()  # Greet the group of people
                last_greeted_time_unknown = time.time()
            elif people_count == 1:
                name = recognized_people[0]
                if name == last_greeted_person:
                    if time.time() - last_greeted_time_person >= 30:
                        greet_person(name)  # Greet the recognized person again after 30 seconds
                        last_greeted_time_person = time.time()
                else:
                    greet_person(name)  # Greet the recognized person
                    last_greeted_person = name
                    last_greeted_time_person = time.time()
            elif people_count == 0 and time.time() - last_greeted_time_unknown >= 10:
                greet_unknown()  # Greet as unknown person
                last_greeted_time_unknown = time.time()
        
        except Exception as e:
            print(f"Error: {e}")
        
        cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('frame', 960, 720)
        cv2.imshow('frame', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    vid.release()
    cv2.destroyAllWindows()

# Run the face recognition function
realtime_face_recognition()


24-07-01 17:15:30 - Searching [[[255 251 253]
  [213 192 190]
  [205 175 168]
  ...
  [ 86  88  74]
  [ 91  95  77]
  [ 99 103  84]]

 [[254 253 247]
  [205 188 179]
  [201 174 163]
  ...
  [ 88  89  79]
  [ 90  92  80]
  [ 90  92  79]]

 [[246 245 238]
  [188 174 165]
  [191 167 155]
  ...
  [ 87  86  82]
  [ 88  88  83]
  [ 84  84  78]]

 ...

 [[ 58  49 100]
  [ 59  49 102]
  [ 58  48 102]
  ...
  [ 57  59  59]
  [116 119 119]
  [131 136 135]]

 [[ 55  49  90]
  [ 57  49  96]
  [ 58  48 102]
  ...
  [ 50  47  47]
  [ 73  69  69]
  [115 111 111]]

 [[ 53  51  80]
  [ 56  52  88]
  [ 58  50  98]
  ...
  [ 48  40  41]
  [ 60  51  52]
  [ 88  78  79]]] in 84 length datastore
24-07-01 17:15:31 - find function duration 0.59902024269104 seconds
24-07-01 17:15:35 - Searching [[[246 250 234]
  [169 162 141]
  [161 149 127]
  ...
  [ 68  66  58]
  [ 67  62  62]
  [ 73  66  71]]

 [[242 241 224]
  [177 162 142]
  [173 152 132]
  ...
  [ 65  66  56]
  [ 66  65  61]
  [ 71  67  68]]

 [[236 233 

24-07-01 17:15:41 - find function duration 0.33003926277160645 seconds
24-07-01 17:15:41 - Searching [[[233 240 216]
  [172 156 132]
  [162 145 120]
  ...
  [ 53  52  55]
  [ 54  55  54]
  [ 56  57  52]]

 [[233 237 211]
  [172 152 127]
  [156 135 109]
  ...
  [ 53  52  54]
  [ 54  55  51]
  [ 56  57  49]]

 [[219 225 196]
  [167 143 119]
  [166 137 115]
  ...
  [ 53  52  52]
  [ 55  55  50]
  [ 55  57  45]]

 ...

 [[ 33  42  39]
  [ 38  42  42]
  [ 45  44  47]
  ...
  [ 65  58  72]
  [ 65  56  70]
  [ 57  47  62]]

 [[ 33  39  36]
  [ 34  41  40]
  [ 38  45  45]
  ...
  [ 79  69  99]
  [ 89  77 106]
  [ 94  80 108]]

 [[ 34  40  36]
  [ 33  43  41]
  [ 34  48  45]
  ...
  [ 83  70 108]
  [ 92  74 112]
  [103  84 120]]] in 84 length datastore
24-07-01 17:15:41 - find function duration 0.33379626274108887 seconds
24-07-01 17:15:41 - Searching [[[246 241 216]
  [176 148 121]
  [186 147 117]
  ...
  [ 56  60  53]
  [ 58  60  55]
  [ 58  59  57]]

 [[241 238 214]
  [170 147 125]
  [171 13

24-07-01 17:15:45 - find function duration 0.34586572647094727 seconds
24-07-01 17:15:45 - Searching [[[240 239 219]
  [167 151 136]
  [166 145 133]
  ...
  [ 52  52  52]
  [ 54  55  53]
  [ 59  60  58]]

 [[232 233 222]
  [162 148 141]
  [161 140 136]
  ...
  [ 54  54  54]
  [ 56  56  56]
  [ 59  59  59]]

 [[220 221 215]
  [155 141 138]
  [157 138 138]
  ...
  [ 55  56  54]
  [ 57  57  57]
  [ 60  60  61]]

 ...

 [[168 177 193]
  [186 195 208]
  [204 214 221]
  ...
  [ 22  22  23]
  [ 24  24  24]
  [ 24  25  23]]

 [[158 172 183]
  [178 191 200]
  [188 202 207]
  ...
  [ 21  20  23]
  [ 23  21  25]
  [ 23  23  25]]

 [[140 161 164]
  [172 190 193]
  [181 198 200]
  ...
  [ 21  19  26]
  [ 22  19  27]
  [ 22  20  26]]] in 84 length datastore
24-07-01 17:15:45 - find function duration 0.33031797409057617 seconds
24-07-01 17:15:45 - Searching [[[234 238 223]
  [173 151 143]
  [155 127 125]
  ...
  [ 54  54  54]
  [ 57  58  56]
  [ 60  61  59]]

 [[228 228 220]
  [168 151 146]
  [158 13

24-07-01 17:15:49 - find function duration 0.3319568634033203 seconds
24-07-01 17:15:52 - Searching [[[249 249 249]
  [226 226 226]
  [214 214 214]
  ...
  [ 66  60  64]
  [ 66  59  65]
  [ 70  60  69]]

 [[255 255 255]
  [244 244 244]
  [220 220 220]
  ...
  [ 63  61  62]
  [ 63  60  64]
  [ 63  59  66]]

 [[253 253 253]
  [255 255 255]
  [239 239 239]
  ...
  [ 62  62  60]
  [ 64  64  64]
  [ 63  61  65]]

 ...

 [[ 99  84 161]
  [ 95  81 154]
  [ 90  78 146]
  ...
  [ 34  34  33]
  [ 30  29  30]
  [ 27  24  31]]

 [[102  90 168]
  [ 94  81 156]
  [ 94  80 153]
  ...
  [ 34  33  37]
  [ 30  30  31]
  [ 28  28  28]]

 [[ 98  92 170]
  [ 91  84 159]
  [ 88  79 152]
  ...
  [ 29  30  33]
  [ 29  31  31]
  [ 28  32  26]]] in 84 length datastore
24-07-01 17:15:52 - find function duration 0.33149194717407227 seconds
24-07-01 17:15:52 - Searching [[[248 248 248]
  [221 221 221]
  [213 213 213]
  ...
  [ 68  67  71]
  [ 66  64  67]
  [ 67  64  66]]

 [[254 254 254]
  [238 238 238]
  [214 214

24-07-01 17:15:56 - find function duration 0.34990429878234863 seconds
24-07-01 17:15:56 - Searching [[[242 244 237]
  [215 218 211]
  [198 199 196]
  ...
  [ 54  49  55]
  [ 54  46  55]
  [ 57  47  58]]

 [[253 255 248]
  [239 241 235]
  [213 214 211]
  ...
  [ 54  47  52]
  [ 55  45  52]
  [ 57  45  54]]

 [[255 255 249]
  [251 253 247]
  [235 237 232]
  ...
  [ 54  48  53]
  [ 56  47  54]
  [ 55  44  54]]

 ...

 [[179 182 181]
  [194 197 196]
  [197 199 199]
  ...
  [ 25  25  25]
  [ 27  26  28]
  [ 27  26  29]]

 [[192 192 192]
  [199 199 199]
  [202 202 202]
  ...
  [ 23  23  23]
  [ 24  24  25]
  [ 25  24  26]]

 [[192 196 195]
  [198 202 201]
  [201 203 203]
  ...
  [ 22  23  21]
  [ 22  22  23]
  [ 22  21  23]]] in 84 length datastore
24-07-01 17:15:56 - find function duration 0.33185338973999023 seconds
24-07-01 17:15:56 - Searching [[[246 246 247]
  [216 215 216]
  [195 195 195]
  ...
  [ 50  49  53]
  [ 51  50  54]
  [ 54  52  58]]

 [[252 254 248]
  [234 236 231]
  [206 20

24-07-01 17:15:59 - find function duration 0.34643983840942383 seconds
24-07-01 17:16:00 - Searching [[[248 251 245]
  [229 230 224]
  [219 217 213]
  ...
  [ 53  53  53]
  [ 53  53  53]
  [ 55  55  55]]

 [[254 254 251]
  [244 244 240]
  [221 218 216]
  ...
  [ 54  54  54]
  [ 54  54  54]
  [ 55  55  55]]

 [[255 255 253]
  [255 255 254]
  [249 246 245]
  ...
  [ 54  54  54]
  [ 53  53  53]
  [ 56  56  56]]

 ...

 [[186 194 199]
  [203 207 210]
  [208 209 210]
  ...
  [ 27  28  25]
  [ 26  27  26]
  [ 28  28  31]]

 [[195 207 207]
  [202 211 211]
  [209 216 215]
  ...
  [ 27  28  28]
  [ 27  27  27]
  [ 26  26  27]]

 [[199 206 205]
  [207 212 211]
  [210 215 214]
  ...
  [ 26  27  31]
  [ 27  28  29]
  [ 24  26  27]]] in 84 length datastore
24-07-01 17:16:00 - find function duration 0.3174269199371338 seconds
24-07-01 17:16:00 - Searching [[[252 252 252]
  [226 226 226]
  [210 210 210]
  ...
  [ 61  62  60]
  [ 61  61  61]
  [ 65  63  64]]

 [[255 255 255]
  [249 249 249]
  [226 226

24-07-01 17:16:10 - find function duration 0.34635448455810547 seconds
24-07-01 17:16:10 - Searching [[[247 248 243]
  [217 218 214]
  [198 199 198]
  ...
  [ 55  52  54]
  [ 54  49  53]
  [ 55  49  54]]

 [[253 254 250]
  [242 243 239]
  [212 213 210]
  ...
  [ 54  51  53]
  [ 52  50  52]
  [ 51  49  51]]

 [[255 255 252]
  [255 255 252]
  [235 238 231]
  ...
  [ 52  52  53]
  [ 50  49  51]
  [ 48  47  49]]

 ...

 [[182 185 192]
  [191 193 198]
  [195 198 200]
  ...
  [ 26  26  26]
  [ 25  25  24]
  [ 24  25  23]]

 [[190 191 194]
  [199 200 202]
  [198 199 200]
  ...
  [ 25  25  25]
  [ 24  25  23]
  [ 24  25  22]]

 [[192 192 192]
  [201 201 201]
  [203 203 203]
  ...
  [ 24  24  25]
  [ 22  22  22]
  [ 22  22  22]]] in 84 length datastore
24-07-01 17:16:10 - find function duration 0.3161742687225342 seconds
24-07-01 17:16:10 - Searching [[[245 247 242]
  [216 217 213]
  [202 203 200]
  ...
  [ 49  48  53]
  [ 52  49  58]
  [ 52  48  59]]

 [[255 255 253]
  [240 241 239]
  [216 217